In [ ]:
#!/usr/bin/env python3

In [4]:
import numpy as np 
import time 
from datetime import timedelta
import tensorflow as tf
from readImages import *
from build_tensorflow_graph import *

In [3]:
#
## ---------- VAE Architecture ----------
#

# Convolution Layer 1

# Convolution Layer 2

# Fully Connected 1

# Lattern Code 
